# 6.5.2 Create Heatmaps for Weather Parameters

In [ ]:

#Import dependencies
import pandas as pd
import citipy
import gmaps
import requests
#Import the API Key
from config import g_key

In [ ]:
#Store the CSV file into a DataFrame
city_data_df = pd.read_csv('weather_data/cities.csv')
city_data_df.head()

In [ ]:
#Get the data types
city_data_df.dtypes

## Create a Maximum Temperature Heatmap

In [ ]:
#Configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [ ]:
#Heatmap of temperature
#Get the latitude and longitude
locations = city_data_df[["Lat", 'Lng']]
#Get the maximum temperature and drop negative temperatures because 'weights' will not allow negative temps
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

#Assign the figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.25)
#Assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False,
                                max_intensity = 300, point_radius=4)
#Performing list comprehension on the heatmap layer
#heat-layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0)
#for temp in max_temp])

#Add the heatmap layer
fig.add_layer(heat_layer)
#Call the figure to plot the data
fig

## Create a Percent Humidity Heatmap

In [ ]:
#Heatmap of percent humidity
locations = city_data_df[['Lat', 'Lng']]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.25)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
#Call the figure to plot the data
fig

In [ ]:
#Heatmap of cloudiness
locations = city_data_df[['Lat', 'Lng']]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.25)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
#Call the figure to plot the data
fig

In [ ]:
#Heatmap of percent wind speed
locations = city_data_df[['Lat', 'Lng']]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.25)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
#Call the figure to plot the data
fig

# Get the Vacation Criteria

In [ ]:
#Ask the customer to add a minimum and maximum temperature value
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))

In [ ]:
#Filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

In [ ]:
preferred_cities_df.count()

# Map Vacation Criteria

## Get Travel Destinations

Don't add the hotel information to the preferred_cities_df DataFrame because this DataFrame is our filtered DataFrame, and the customer will always filter it for each trip. We'll need to create a new DataFrame specifically for the data needed to create a heatmap and pop-up markers.

In [ ]:
#Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[['City', 'Country',  'Max Temp', "Lat",  "Lng", ]].copy()
hotel_df["Hotel Name"] = ''
hotel_df.head(10)

## Retrieve Hotels from a Nearby Search

Review the Nearby Search Results on the Google Place Search page here: https://developers.google.com/maps/documentation/places/web-service/search#PlaceSearchRequests%0D%0A

In [ ]:
#Dependencies and Setup
import requests
import gmaps

#Import API key
from config import g_key

In [ ]:
#Set paramaters to search for a hotel in Paris
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key,
    'location': '48.8566, 2.3522'}
#Iterate through the DataFrame
for index, row in hotel_df.iterrows():
    #Get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    #Add the latitude and longitude to location key for the params dictionary
    params['location'] = f'{lat}, {lng}'
    
    #Use the search term: 'lodging' and our lat and lng
    #Use base URL to search for hotels in Paris
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #Make request and get the JSON data from teh search
    hotels = requests.get(base_url, params=params).json()
    #Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')

hotel_df.head(10)

## Create a Maximum Temperature Heatmap from a Hotel DataFrame

In [ ]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig


In [ ]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} &deg;F</dd>
</dl>
"""

In [ ]:
#Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [ ]:
#Add a heatmap of temperature for the vacation spots and a pop-up marker for each city
locations = hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

#Call the figure to plot the data
fig